In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pprint import pprint
from dotenv import load_dotenv
from tqdm import tqdm
from pathlib import Path

from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    AIMessage,
    RemoveMessage,
    ToolMessage
)


root_dir = Path(os.getcwd()).parent.parent
sys.path.insert(0, str(root_dir))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [ ]:
from src.d01_data.graph import graph

raw_path = root_dir / 'data' / '01_raw'
intermediate_path = root_dir / 'data' / '02_intermediate'
memories_path = root_dir / 'data' / '03_memories'
output_path = root_dir / 'data' / '04_model_output'

In [ ]:
# import sqlite3
# from src.d01_data.data import create_usuarios_table, get_user_info, upsert_user

# long_term_db_path = memories_path / 'long_term_memory.db'
# long_term_db_conn = sqlite3.connect(str(long_term_db_path), check_same_thread=False)
# create_usuarios_table(long_term_db_conn)

# # Example usage:
# user_info = get_user_info(long_term_db_conn, 3)
# print("User Info:", user_info)

In [ ]:
# graph

In [ ]:
user_id = 367
config = {'configurable': {'thread_id': str(user_id), 'user_id':str(user_id)}}

while True:
    user_input = input('Enter your message (type "quit" to exit): ')
    if user_input.lower() == 'quit':
        print('Exiting the conversation. Goodbye!')
        break
    
    input_message = HumanMessage(content=user_input)
    outputs = graph.invoke({'messages': [input_message]}, config, stream_mode='updates')
    
    for out in outputs:
        node_name = list(out.keys())[0]
        
        if isinstance(out[node_name]['messages'], list):
            for mess in out[node_name]['messages']:
                mess.pretty_print()
        else:
            out[node_name]['messages'].pretty_print()


================================== Ai Message ==================================

Puedo ayudarte a buscar documentos legales en la base de datos legal española. Proporciona una consulta en lenguaje natural y puedo buscar documentos relevantes del Código Civil, Código Penal o la Constitución Española. También puedes controlar el equilibrio entre la búsqueda de palabras clave y la búsqueda semántica utilizando el parámetro alfa.
================================= Tool Message =================================

No hay nuevos datos personales que actualizar para el usuario 6
================================== Ai Message ==================================
Tool Calls:
  consult_legal_database (28d20366-cd25-434b-85b2-11e9137e5fdf)
 Call ID: 28d20366-cd25-434b-85b2-11e9137e5fdf
  Args:
    filter: {'origen': {'$in': ['Constitucion Española']}}
    query: articulo 65 de la constitucion española
    alpha: 0.1
================================= Tool Message =================================
Name:

In [14]:
for out in outputs:
        node_name = list(out.keys())[0]
        
        if isinstance(out[node_name]['messages'], list):
            for mess in out[node_name]['messages']:
                mess.pretty_print()
        else:
            out[node_name]['messages'].pretty_print()

================================== Ai Message ==================================

No sé cómo te llamas. No tengo acceso a información personal.
================================= Tool Message =================================

No hay nuevos datos personales que actualizar para el usuario 1


In [13]:
outputs

[{'conversation': {'messages': AIMessage(content='No sé cómo te llamas. No tengo acceso a información personal.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-e2f5c12b-7e82-421f-8244-44050cc1c27d-0', usage_metadata={'input_tokens': 539, 'output_tokens': 13, 'total_tokens': 552, 'input_token_details': {'cache_read': 0}})}},
 {'update_memory': {'messages': [ToolMessage(content='No hay nuevos datos personales que actualizar para el usuario 1', id='0b3b85b4-2127-4de8-a578-e6a07b7056e9', tool_call_id='67438012-42fb-475c-97cc-c78c4fa60f84')]}}]

In [99]:
from src.d00_utils.utils import parse_graph_output

In [108]:
tmp = parse_graph_output(outputs, debug=True)

content='Lo siento, no he podido encontrar el artículo 2 del Código Civil. La búsqueda me ha devuelto otros artículos pero no el que solicitaste. ¿Puedo ayudarte con alguna otra cosa?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-80404c55-56df-4014-89b9-25cc5c1c09f1-0' usage_metadata={'input_tokens': 3573, 'output_tokens': 38, 'total_tokens': 3611, 'input_token_details': {'cache_read': 0}}


In [110]:
formatted_messages = []
for tag, content in tmp:
    # escaped_content = html.escape(content)
    if tag in ['fn', 'tool']:
        # Wrap messages in green text
        formatted_messages.append(f"<span style='color: green;'>{content}</span>")
    else:
        formatted_messages.append(content)

In [ ]:
for t in formatted_messages:
    print(t)

In [58]:
config = {'configurable': {'thread_id': '1'}}
graph_state = graph.get_state(config)
print(graph_state)

StateSnapshot(values={'messages': [HumanMessage(content='Sabes como me llamo?', additional_kwargs={}, response_metadata={}, id='05ec9be9-bb95-42d3-9edf-9beec7a8a184'), AIMessage(content='No te conozco, no sé cómo te llamas.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-f2eb9e5b-5196-4aef-b99b-e50962a9d260-0', usage_metadata={'input_tokens': 806, 'output_tokens': 10, 'total_tokens': 816, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='Me llamo Edgar para que lo sepas', additional_kwargs={}, response_metadata={}, id='dea8a0c5-5c3a-49c2-a177-9269fcbda933'), AIMessage(content='Gracias, Edgar. Es un placer conocerte.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-afb0d8b3-1c87-4409-b10e-15626e854777-0', usage_metadata={'input_tokens': 824, 'output_

In [59]:
graph_state

StateSnapshot(values={'messages': [HumanMessage(content='Sabes como me llamo?', additional_kwargs={}, response_metadata={}, id='05ec9be9-bb95-42d3-9edf-9beec7a8a184'), AIMessage(content='No te conozco, no sé cómo te llamas.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-f2eb9e5b-5196-4aef-b99b-e50962a9d260-0', usage_metadata={'input_tokens': 806, 'output_tokens': 10, 'total_tokens': 816, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='Me llamo Edgar para que lo sepas', additional_kwargs={}, response_metadata={}, id='dea8a0c5-5c3a-49c2-a177-9269fcbda933'), AIMessage(content='Gracias, Edgar. Es un placer conocerte.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-afb0d8b3-1c87-4409-b10e-15626e854777-0', usage_metadata={'input_tokens': 824, 'output_

In [60]:
graph_state

StateSnapshot(values={'messages': [HumanMessage(content='Sabes como me llamo?', additional_kwargs={}, response_metadata={}, id='05ec9be9-bb95-42d3-9edf-9beec7a8a184'), AIMessage(content='No te conozco, no sé cómo te llamas.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-f2eb9e5b-5196-4aef-b99b-e50962a9d260-0', usage_metadata={'input_tokens': 806, 'output_tokens': 10, 'total_tokens': 816, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='Me llamo Edgar para que lo sepas', additional_kwargs={}, response_metadata={}, id='dea8a0c5-5c3a-49c2-a177-9269fcbda933'), AIMessage(content='Gracias, Edgar. Es un placer conocerte.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-afb0d8b3-1c87-4409-b10e-15626e854777-0', usage_metadata={'input_tokens': 824, 'output_

In [61]:
graph_state

StateSnapshot(values={'messages': [HumanMessage(content='Sabes como me llamo?', additional_kwargs={}, response_metadata={}, id='05ec9be9-bb95-42d3-9edf-9beec7a8a184'), AIMessage(content='No te conozco, no sé cómo te llamas.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-f2eb9e5b-5196-4aef-b99b-e50962a9d260-0', usage_metadata={'input_tokens': 806, 'output_tokens': 10, 'total_tokens': 816, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='Me llamo Edgar para que lo sepas', additional_kwargs={}, response_metadata={}, id='dea8a0c5-5c3a-49c2-a177-9269fcbda933'), AIMessage(content='Gracias, Edgar. Es un placer conocerte.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-afb0d8b3-1c87-4409-b10e-15626e854777-0', usage_metadata={'input_tokens': 824, 'output_